In [1]:
import pandas as pd
import numpy as np
import censusgeocode as cg
from random import uniform
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm

In [2]:
crashes = pd.read_csv('/Users/hank/Documents/UChicago/Data Engineering Platforms/Final Project/bike_crashes_clean.csv')
divvystations = pd.read_csv('/Users/hank/Documents/UChicago/Data Engineering Platforms/Final Project/Divvy_Bicycle_Stations_20231113.csv')
bikeracks = pd.read_csv('/Users/hank/Documents/UChicago/Data Engineering Platforms/Final Project/Bike_Racks.csv')

In [3]:
#cg.coordinates(x=-87, y=42)

In [4]:
bikeracks_coord = bikeracks[['Latitude','Longitude']].rename(columns={"Latitude": "latitude", "Longitude": "longitude"})
divvystations_coord = divvystations[['Latitude','Longitude']].rename(columns={"Latitude": "latitude", "Longitude": "longitude"})
crashes_coord = crashes[['LATITUDE','LONGITUDE']].rename(columns={"LATITUDE": "latitude", "LONGITUDE": "longitude"})

In [5]:
combined_df = pd.concat([bikeracks_coord, divvystations_coord, crashes_coord], ignore_index=True)
combined_df['latitude'] = combined_df['latitude'].round(4)
combined_df['longitude'] = combined_df['longitude'].round(4)
combined_df = combined_df.drop_duplicates().dropna(how='all')

In [6]:
#geo_list = []
#n = 0 
#for f, g in zip(combined_df['latitude'], combined_df['longitude']):
#    n = n + 1
#    if n == 10: 
#        break
#    try:
#        census = cg.coordinates(x=g, y=f)['2020 Census Blocks'][0]
#        data1 = [census['STATE'],census['COUNTY'],census['TRACT'], census['BLKGRP'], f, g]
#        geo_list.append(data1)
#    except:
#        pass
#    

In [7]:
#geo_list

In [8]:
#combined_df = combined_df.head(100)

In [9]:
def geocode(row):
    index, lat, lng = row
    try:
        census = cg.coordinates(lng, lat)['2020 Census Blocks'][0]

        data = dict(geoid=census['GEOID'], 
                    state=census['STATE'], 
                    county=census['COUNTY'], 
                    tract=census['TRACT'], 
                    blockgrp=census['BLKGRP'], 
                    lat=lat, 
                    lng=lng)

    except Exception as e:
        data = dict(lat=lat, 
                    lng=lng)

    return data

with ThreadPoolExecutor() as tpe:
     data = list(tqdm(tpe.map(geocode, combined_df.itertuples()), total=len(combined_df)))
df = pd.DataFrame.from_records(data)

  0%|          | 0/13630 [00:00<?, ?it/s]

In [16]:
df['location_id'] = range(1, len(df) + 1)
intcols = ['geoid', 'state','county','tract','blockgrp'] 

In [17]:
df = df.dropna()
df[cols] = df[intcols].astype(int)

In [19]:
df.to_csv('/Users/hank/Documents/UChicago/Data Engineering Platforms/Final Project/Location_Mapping.csv')